In [2]:
import os
import pickle as pkl

from sklearn.utils import resample
import matplotlib.pyplot as plt
from keras.preprocessing.image import array_to_img
import random 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from IPython.display import clear_output

ROOT_DIR = '/content/drive/My Drive/NucleiSegmentation'
os.chdir(ROOT_DIR)

import sys
sys.path.append(os.path.join(ROOT_DIR, "src")) 
from datagen import DataGenerator
from models import residual_attentionunet
from evaluation_metrics import *

with open(os.path.join(ROOT_DIR,"data","processed","MOSID.dat"),'rb') as f : dataset = pkl.load(f)

In [3]:
X_train, y_train = dataset['Training']['TissueImages'], dataset['Training']['GroundTruth']
X_val, y_val = dataset['Validation']['TissueImages'], dataset['Validation']['GroundTruth']

In [4]:
NUM_MODELS = 33
BATCH_SIZE = 8
MAX_EPOCHS = 30

WEIGHTS_PATH = os.path.join(ROOT_DIR,"models","weights", "AttentionResidualUNet")

In [ ]:
for num in range(NUM_MODELS):
  
  clear_output(wait=True)
  print("Training model #" + str(num))
  
  X_sample, y_sample = resample(X_train, y_train, replace = True, n_samples = int(2/3 * len(X_train)))
  
  datagen = DataGenerator(seed = 10, X_fit = X_sample, y_fit = y_sample)
  train_generator = datagen.get_generator(X = X_sample, y = y_sample, batch_size = BATCH_SIZE)
  val_generator = datagen.get_generator(X = X_val, y = y_val, batch_size = BATCH_SIZE)
  train_steps, val_steps = len(X_sample) / BATCH_SIZE, len(X_val) / BATCH_SIZE

  model = residual_attentionunet(input_shape = (256, 256, 3))
  model.compile(optimizer='adam', loss= IoU_loss, metrics=['accuracy', IoU_coef, dice_coef, f1])

  checkpoint_path = WEIGHTS_PATH + "/model-" + str(num) +"-best.h5"

  callbacks = [
      EarlyStopping(patience=5, verbose=1, monitor = "val_IoU_coef", mode = "max"),
      ModelCheckpoint(checkpoint_path, verbose=1, save_best_only=True, save_weights_only=False, monitor = "val_IoU_coef", mode = "max")
    ]
  
  model.fit(train_generator, validation_data = val_generator, 
          epochs = MAX_EPOCHS, steps_per_epoch = train_steps, validation_steps = val_steps, callbacks = callbacks) 

In [22]:
import numpy as np
import cv2

def ensemble_preds(pred_masks):
  n = len(pred_masks)
  fusion = np.sum(pred_masks, axis=0)
  return np.where(fusion >= n / 2, 1, 0)

def augment(img):
  augmentations = []
  augmentations.append(img)
  augmentations.append(cv2.flip(img, 0))
  augmentations.append(cv2.flip(img, 1))
  augmentations.append(cv2.flip(img, -1))
  augmentations.append(cv2.rotate(img, cv2.cv2.ROTATE_90_CLOCKWISE))
  augmentations.append(cv2.rotate(img, cv2.ROTATE_180))
  augmentations.append(cv2.rotate(img, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE))
  return np.array(augmentations), len(augmentations)

def undo_augmentations(imgs):
  origs = []
  origs.append(imgs[0])
  origs.append(cv2.flip(imgs[1],0))
  origs.append(cv2.flip(imgs[2],1))
  origs.append(cv2.flip(imgs[3],-1))
  origs.append(cv2.rotate(imgs[4], cv2.cv2.ROTATE_90_COUNTERCLOCKWISE))
  origs.append(cv2.rotate(imgs[5], cv2.cv2.ROTATE_180))
  origs.append(cv2.rotate(imgs[6], cv2.cv2.ROTATE_90_CLOCKWISE))
  for i in range(1,len(origs)): origs[i] = np.expand_dims(origs[i], axis=2)
  return np.array(origs)

In [14]:
models = []

custom_objs = {
    'IoU_coef' : IoU_coef,
    'dice_coef' : dice_coef,
    'f1' : f1
}

NUM_MODELS = 33

for num in range(NUM_MODELS):
  model_path = os.path.join(ROOT_DIR, "models", "weights","AttentionResidualUNet","model-" + str(num) + "-best.h5")
  models.append(load_model(model_path, custom_objects= custom_objs, compile = False))

In [ ]:
import pandas as pd 

ious, f1s = [], []
for model in models:
  prediction = model.predict(X_test)
  ious.append(IoU_coef(y_test, prediction).numpy())
  f1s.append(f1(y_test, prediction).numpy())

df = pd.DataFrame({'IoU' : ious, 'f1' : f1s})

In [53]:
sum(f1s)/len(f1s)

0.8156406716866926

In [ ]:
from tqdm.notebook import tqdm

X_test, y_test = dataset['Testing']['TissueImages'], dataset['Testing']['GroundTruth']

y_pred = []

for i in tqdm(range(len(X_test))):
  model_inputs, num_augmentations = augment(X_test[i])
  model_preds, preds = [], []
  for model in models: model_preds.append(np.round(model.predict(model_inputs)))
  model_preds = np.array(model_preds)
  for i in range(num_augmentations):
    aug_preds = []
    for j in range(NUM_MODELS): aug_preds.append(model_preds[j][i])
    preds.append(ensemble_preds(np.array(aug_preds)))
  y_pred.append(ensemble_preds(undo_augmentations(np.array(preds))))

y_pred = np.array(y_pred).astype('float32')

In [ ]:
y_pred_alt = []

for i in tqdm(range(len(X_test))):
  model_inputs, num_augmentations = augment(X_test[i])
  model_preds, preds = [], []
  for model in models: model_preds.append(np.round(model.predict(model_inputs)))
  model_preds = np.array(model_preds)
  for i in range(NUM_MODELS): preds.append(ensemble_preds(undo_augmentations(model_preds[i])))
  y_pred_alt.append(ensemble_preds(preds)) 

In [48]:
y_pred_alt = np.array(y_pred_alt).astype('float32')

In [30]:
# Watershed 
def watershed(pred_mask):
  mask = np.uint8(pred_mask[:, :, 0])
  ret1, thresh = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  kernel = np.ones((3, 3), np.uint8)
  opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
  sure_bg = cv2.dilate(opening, kernel, iterations=10)
  dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
  ret2, sure_fg = cv2.threshold(dist_transform, 0.25*dist_transform.max(), 255, 0)
  sure_fg = np.uint8(sure_fg)
  unknown = cv2.subtract(sure_bg, sure_fg)
  ret3, markers = cv2.connectedComponents(sure_fg)
  markers = markers + 10
  markers[unknown==255] = 0
  pred_mask_3c = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
  markers = cv2.watershed(pred_mask_3c, markers)
  pred_mask_3c[markers == -1] = [0, 255, 255] 
  return pred_mask_3c[:, :, 0]
  
watershed_preds = []
for i in range(len(y_pred)):
  watershed_preds.append(np.expand_dims(watershed(y_pred[i]), axis = 2))

In [49]:
print("Accuracy : ", accuracy(y_test, y_pred_alt))
print("Dice : ", dice_coef(y_test, y_pred_alt).numpy())
print("IoU : ", IoU_coef(y_test, y_pred_alt).numpy())
print("F1 : ", f1(y_test, y_pred_alt).numpy())

Accuracy :  0.9116260740492079
Dice :  0.82540154
IoU :  0.70270956
F1 :  0.8254014


In [50]:
print("Accuracy : ", accuracy(y_test, y_pred))
print("Dice : ", dice_coef(y_test, y_pred))
print("IoU : ", IoU_coef(y_test, y_pred))
print("F1 : ", f1(y_test, y_pred))

Accuracy :  0.9116027620103624
Dice :  tf.Tensor(0.82539386, shape=(), dtype=float32)
IoU :  tf.Tensor(0.70269847, shape=(), dtype=float32)
F1 :  tf.Tensor(0.8253937, shape=(), dtype=float32)
